In [94]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd

import os
import sys
from pathlib import Path

sys.path.insert(0, "../scripts")

from utils import train_test_split, image_batch_generator, get_train_augmentation, get_table_augmentation
from utils import DATASET_PATH, DS_IMAGES, DS_MASKS
import utils
from models import TableNet, load_unet_model
import models
from vis import imshow, anshow, colanshow
import vis
import metrics

import importlib
importlib.reload(utils)
importlib.reload(vis)
importlib.reload(models)
importlib.reload(metrics)

<module 'metrics' from '/home/cseadmin/Tigran/table_extractor/notebooks/../scripts/metrics.py'>

In [70]:
DS_DIR = "../datasets/Images_coordinates/"

In [43]:
DS_IMAGES = list(Path(DS_DIR).glob("*.jpg"))

In [22]:
ann_df = pd.read_csv(DS_DIR + "/Coordinates.csv")
ann_df.head()

,Name,x_min,y_min,x_max,y_max
0,0803.1220.pdf_page_0.jpg,330,1314,1378,1453
1,0803.1220.pdf_page_0.jpg,336,1657,1369,1917
2,0803.1220.pdf_page_1.jpg,345,345,1360,1078
3,Alcester_Grammar_School_The_Record_157_1991_ A...,259,200,978,1412
4,Alcester_Grammar_School_The_Record_157_1991_ A...,124,222,922,1204


In [10]:
img = cv2.imread(os.path.join(DS_DIR, DS_IMAGES[0]))

In [64]:
# ann_df[ann_df["Name"] == "mtrl-th9505007.pdf_page_38.jpg"]

,Name,x_min,y_min,x_max,y_max


In [68]:
for img_name in DS_IMAGES:
    img = cv2.imread(str(img_name), cv2.IMREAD_GRAYSCALE)
    annotations = ann_df[ann_df["Name"] == img_name.name]
    if len(annotations) == 0:
        continue

    mask = np.zeros_like(img)
    # print(annotations)
    for k, v in annotations.iterrows():
        # print(k, v)
        y1, y2, x1, x2 = int(v["y_min"]), int(v["y_max"]), int(v["x_min"]), int(v["x_max"])
        mask[y1:y2, x1:x2] = 255

    # print(img_name)
    # img = img / 255
    # mask = mask / 255

    # colanshow(img, mask, fac=200)
    b_n, ext = os.path.splitext(img_name.name)
    cv2.imwrite(f"../datasets/ann_images/{img_name.name}", img)
    cv2.imwrite(f"../datasets/ann_masks/{b_n}_mask{ext}", mask)
    # break

In [50]:
img_name = DS_IMAGES[0]
print(str(img_name))
img = cv2.imread(str(img_name))
annotations = ann_df[ann_df["Name"] == img_name.name]
# print(annotations)
for k, v in annotations.iterrows():
    print(v)

../datasets/Images_coordinates/0708.1766.pdf_page_18.jpg
Name     0708.1766.pdf_page_18.jpg
x_min                          288
y_min                          230
x_max                         1408
y_max                         1952
Name: 11, dtype: object
Name     0708.1766.pdf_page_18.jpg
x_min                          292
y_min                         1985
x_max                         1405
y_max                         2073
Name: 12, dtype: object


## Predicted on annotated dataset.

In [71]:
ANN_IMAGE_NAMES = os.listdir("../datasets/ann_images/")

In [84]:
model = load_unet_model((512, 512), 2)
checkpoint = tf.train.Checkpoint(step=tf.Variable(1), optimizer=tf.keras.optimizers.Adam(), net=model)
status = checkpoint.restore("../scripts/training_checkpoints/" + '2022.07.30-22/' + 'ckpt-238')
status

In [88]:
loss_fn = metrics.jaccard_distance

tr_metrics = {n:[] for n in ("loss", "iou", "f1", "precision", "recall")}

for i, (batch_X, batch_y) in enumerate(image_batch_generator(
                            ANN_IMAGE_NAMES, 8, (512, 512), include_edges_as_band=True, normalize=True
                        )):

    # colanshow(batch_X[0][:, :, 0], batch_y[0], fac=50)
    logits = model(batch_X, training=False)
    logits = tf.squeeze(logits)

    loss_value = loss_fn(batch_y, logits)

    (
        iou_value, f1_score_value, 
        presicion_value, 
        recall_value
    ) = metrics.calculate_metrics(batch_y, logits)
    tr_metrics["loss"].append(np.mean(loss_value))
    tr_metrics["iou"].append(iou_value)
    tr_metrics["f1"].append(f1_score_value)
    tr_metrics["precision"].append(presicion_value)
    tr_metrics["recall"].append(recall_value)

    print(f"{i+1}/{len(ANN_IMAGE_NAMES)//8}", end='\r')
    if i + 1 >= len(ANN_IMAGE_NAMES) // 8:
        break

In [97]:
utils.save_pred_samples(
        model, ANN_IMAGE_NAMES, (512, 512), 1,
        "test", directory=f"./predicted_samples/test_name"
    )

In [90]:
"loss {:.4f}, iou {:.4f}, f1 {:.4f}, precision {:.4f}, recall {:.4f}, ".format(
    np.mean(tr_metrics["loss"]), 
    np.mean(tr_metrics["iou"]), 
    np.mean(tr_metrics["f1"]), np.mean(tr_metrics["precision"]), np.mean(tr_metrics["recall"]))

'loss 19.0365, iou 0.5453, f1 0.6606, precision 0.8348, recall 0.6184, '